In [1]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


In [2]:
train = pd.read_csv("train_E6oV3lV.csv")

In [3]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [4]:
train.shape

(31962, 3)

In [5]:
from keras.preprocessing.text import Tokenizer
t = Tokenizer()

In [6]:
t.fit_on_texts(train['tweet'])

In [7]:
x = t.texts_to_matrix(train['tweet'], mode= 'count')

In [8]:
from sklearn.preprocessing import LabelEncoder
number = LabelEncoder()

In [9]:
y = train.label

In [10]:
x.shape

(31962, 45890)

In [11]:
x[1:4,:]

array([[ 0.,  2.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])

In [12]:
max_review_length = 160
X_train = sequence.pad_sequences(x, maxlen=max_review_length)

In [13]:
X_train.shape

(31962, 160)

In [14]:
import keras.backend as K

def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [15]:
# create the model
embedding_vecor_length = 32
top_words = 500
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', recall])

In [ ]:
model.summary()
model.fit(X_train, y, epochs=1, batch_size=100, validation_split = 0.4)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 160, 32)           16000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 69,301
Trainable params: 69,301
Non-trainable params: 0
_________________________________________________________________
Train on 19177 samples, validate on 12785 samples
Epoch 1/1
14100/19177 [=====================>........] - ETA: 26s - loss: 0.2835 - acc: 0.9253 - recall: 0.0071

In [ ]:
#### Final evaluation of the model
scores = model.evaluate(X_train, y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))